# Spatial Joining with fiona and shapely

## Notes 

- Joins longitude and latitude points from a pandas dataframe to shape files polygons (and properties).
- Uses ERSI postcode and lga shape files from the ABS website.
- Includes postcode and local government area options.
- Accuracy can be traded off for speed using the tolerance_setting parameter.


## Importing modules

In [1]:
import fiona
import pandas
from shapely.geometry import Point, asShape
import datetime as dt
import numpy
from shapely import speedups
speedups.enable()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Set option for lga or postcode
Note: Shapefiles contain polygon boundaries from 2016, take caution when joining data preceding this time

In [2]:
lga = True
postcode = False

In [3]:
#Default to LGA if both LGA and Postcode are selected.

if lga == False and postcode == False:
    lga = True
    
if lga == True and postcode == True:
    postcode = False

# Conditionally set shape file paths.

if postcode == True:
    shapefile_path = "/users/danielcorcoran/desktop/github_repos/python_nb_data_spatial/data/POA_16/POA_2016_AUST.shp"
    property_code_name = "POA_CODE16"
elif lga == True:
    shapefile_path = "/users/danielcorcoran/desktop/github_repos/python_nb_data_spatial/data/LGA_ABS_16/LGA_2016_AUST.shp"
    property_code_name = "LGA_CODE16" 

## Set input path for file containing coordinates, and export path

In [6]:
#note: the input path is the export path in this case
input_path = "/users/danielcorcoran/Desktop/test_coordinates_10K.csv"
export_path = "/users/danielcorcoran/Desktop/"

## Import data

In [7]:
coord_data = pandas.read_csv(input_path)

## Set column names containing longitude and latitude values 
Check the headers here and set the longitude and latitude column names below

In [8]:
coord_data.head()

,longitude,latitude,id
0,144.200311,-36.814039,unique_0
1,144.296159,-32.650504,unique_1
2,147.628548,-38.126470,unique_2
3,144.437691,-37.149164,unique_3
4,147.175428,-37.748786,unique_4


In [9]:
latitude_column_name = "latitude"
longitude_column_name = "longitude"

## Iterate through fiona collection and store polygons, simplified polygons, properties and boundaries
Creating reference data in the form of lists

In [10]:
#Note: Code is BUILT TO FIT the ABS LGA/Postcode shapefiles released in 2016. 
#Take caution when using data from preceding 2016.

fiona_collection = fiona.open(shapefile_path)
fiona_collection_list = list(fiona_collection)

shapes_list = []
simple_shapes_list = []
shapes_properties_list = []
shapes_bounds_list = []

tolerance_setting = 0.0000005

for index in range(len(fiona_collection_list)):
    
    if lga == True and postcode == False:
        current_shape = fiona_collection_list[index]
        state_name = current_shape["properties"]["STE_NAME16"]
        properties = current_shape["properties"]

        if state_name == "Victoria" and current_shape["geometry"] is not None:
            shapes_list.append(asShape(current_shape["geometry"]))
            simple_shapes_list.append(asShape(current_shape["geometry"]).simplify(tolerance = tolerance_setting))
            shapes_properties_list.append(current_shape["properties"])
            shapes_bounds_list.append(asShape(current_shape["geometry"]).bounds)
            
    elif lga == False and postcode == True:    
        current_shape = fiona_collection_list[index]
        postal_state_code = list(current_shape["properties"]["POA_CODE16"])[0]
        properties = current_shape["properties"]

        if postal_state_code == "3" and current_shape["geometry"] is not None:
            shapes_list.append(asShape(current_shape["geometry"]))
            simple_shapes_list.append(asShape(current_shape["geometry"]).simplify(tolerance = tolerance_setting))
            shapes_properties_list.append(current_shape["properties"])
            shapes_bounds_list.append(asShape(current_shape["geometry"]).bounds)

## Check the size of each list
Checking how many polygons we have in total, there should be 698 postcode polygons, or 80 lga polygons

In [11]:
len(shapes_list)

80

In [12]:
len(shapes_bounds_list)

80

In [13]:
len(simple_shapes_list)

80

In [14]:
len(shapes_properties_list)

80





## Calculate the master bounds
master_bounds corresponds to maximum and minimum coordinates for Victoria. These coordinates are required to speed up algorithm time.

In [15]:
master_bounds = [shapes_bounds_list[0][0], shapes_bounds_list[0][1],
shapes_bounds_list[0][2],shapes_bounds_list[0][3]]

In [16]:
for shape_bounds_tuple in shapes_bounds_list:
    
    current_long_low = shape_bounds_tuple[0]
    current_long_high = shape_bounds_tuple[2]
    current_lat_low = shape_bounds_tuple[1]
    current_lat_high = shape_bounds_tuple[3]
    
    master_long_low = master_bounds[0]
    master_long_high = master_bounds[2]
    master_lat_low = master_bounds[1]
    master_lat_high = master_bounds[3]
    
    if current_long_low < master_long_low:
        master_bounds[0] = current_long_low
        print("Master bounds longitude (low) has been updated to",master_bounds[0])
    
    if current_long_high > master_long_high:
        master_bounds[2] = current_long_high
        print("Master bounds longitude (high) has been updated to",master_bounds[2])
        
    if current_lat_low < master_lat_low:
        master_bounds[1] = current_lat_low
        print("Master bounds latitude (low) has been updated to",master_bounds[1])
        
    if current_lat_high > master_lat_high:
        master_bounds[3] = current_lat_high
        print("Master bounds latitude (high) has been updated to",master_bounds[3])

Master bounds longitude (low) has been updated to 142.353480213
Master bounds latitude (low) has been updated to -37.836583308999934
Master bounds latitude (low) has been updated to -38.67879995399994
Master bounds latitude (high) has been updated to -36.25505162799993
Master bounds latitude (high) has been updated to -35.19282674999994
Master bounds latitude (low) has been updated to -38.85769295599994
Master bounds longitude (high) has been updated to 149.9762909950001
Master bounds longitude (low) has been updated to 140.965735138
Master bounds longitude (low) has been updated to 140.96168199600004
Master bounds latitude (high) has been updated to -33.98042558699996
Master bounds latitude (low) has been updated to -39.13674005499996
Master bounds latitude (low) has been updated to -39.159189527999956


In [17]:
#Check master bounds

master_bounds

[140.96168199600004,
 -39.159189527999956,
 149.9762909950001,
 -33.98042558699996]

## Dictionary to store results

In [18]:
results = {"data_row_index":[],
           "matched_code":[]}

## Process, output results

In [20]:
#Reset counter, contains every coordinate match against a shapefile polygon
contains = 0

#Start timer
start = dt.datetime.now()

#Iterate through coordinate data set, upper level loop
for data_index in range(coord_data.shape[0]):
    
    print("Starting check for geopoint #",data_index)
    
    longitude = coord_data.loc[data_index, longitude_column_name]
    latitude = coord_data.loc[data_index, latitude_column_name]
    point = Point(longitude, latitude)
    
    #Check if point is within the maximum bounds for victoria
    if master_bounds[0] <= longitude <= master_bounds[2] and master_bounds[1] <= latitude <= master_bounds[3]:

        #If within Victorias bounds, start checking against each lga polygon
        for polygon_index in range(len(shapes_bounds_list)):

                #Check if point is within maximum bounds for LGA
                if shapes_bounds_list[polygon_index][0] <= longitude <= shapes_bounds_list[polygon_index][2] and shapes_bounds_list[polygon_index][1] <= latitude <= shapes_bounds_list[polygon_index][3]:

                    #If within LGA box boundary, check if the polygon contains point
                    #simple_shapes_list contains 'reduced' lga polygons
                    if simple_shapes_list[polygon_index].contains(point):

                        print("\t>Coordinates at data index",data_index,"contains point")

                        results["data_row_index"].append(data_index)
                        results["matched_code"].append(shapes_properties_list[polygon_index][property_code_name])
                        contains = contains + 1
                        
                        break

    else:
        print("\t>Shape at data index",data_index,"is outside of Victoria's bounds.")

#Show points found and time taken to run
end = dt.datetime.now()
difference_time = end - start

timestamp = end.strftime("%y%m%d_%H%M")

print("\n","POINTS FOUND:",contains)
print("POINTS TOTAL:",coord_data.shape[0])
print("FINISHED IN:",difference_time)

#Optional : export spine file
#result_dataframe = pandas.DataFrame(results)
#result_dataframe.to_csv("sjoin_output_"+timestamp+".csv", index = False)
#print("Spine data was exported successfully. Use this dataset to join the original data to shape files.")

Starting check for geopoint # 0
	>Coordinates at data index 0 contains point
Starting check for geopoint # 1
	>Shape at data index 1 is outside of Victoria's bounds.
Starting check for geopoint # 2
Starting check for geopoint # 3
	>Coordinates at data index 3 contains point
Starting check for geopoint # 4
	>Coordinates at data index 4 contains point
Starting check for geopoint # 5
	>Coordinates at data index 5 contains point
Starting check for geopoint # 6
	>Coordinates at data index 6 contains point
Starting check for geopoint # 7
	>Coordinates at data index 7 contains point
Starting check for geopoint # 8
	>Coordinates at data index 8 contains point
Starting check for geopoint # 9
	>Coordinates at data index 9 contains point
Starting check for geopoint # 10
	>Coordinates at data index 10 contains point
Starting check for geopoint # 11
	>Shape at data index 11 is outside of Victoria's bounds.
Starting check for geopoint # 12
	>Shape at data index 12 is outside of Victoria's bounds.
St

Starting check for geopoint # 196
	>Shape at data index 196 is outside of Victoria's bounds.
Starting check for geopoint # 197
Starting check for geopoint # 198
	>Coordinates at data index 198 contains point
Starting check for geopoint # 199
	>Shape at data index 199 is outside of Victoria's bounds.
Starting check for geopoint # 200
Starting check for geopoint # 201
Starting check for geopoint # 202
	>Coordinates at data index 202 contains point
Starting check for geopoint # 203
	>Coordinates at data index 203 contains point
Starting check for geopoint # 204
Starting check for geopoint # 205
	>Coordinates at data index 205 contains point
Starting check for geopoint # 206
Starting check for geopoint # 207
	>Coordinates at data index 207 contains point
Starting check for geopoint # 208
	>Coordinates at data index 208 contains point
Starting check for geopoint # 209
	>Shape at data index 209 is outside of Victoria's bounds.
Starting check for geopoint # 210
	>Coordinates at data index 210

Starting check for geopoint # 369
	>Coordinates at data index 369 contains point
Starting check for geopoint # 370
Starting check for geopoint # 371
Starting check for geopoint # 372
	>Shape at data index 372 is outside of Victoria's bounds.
Starting check for geopoint # 373
	>Shape at data index 373 is outside of Victoria's bounds.
Starting check for geopoint # 374
	>Shape at data index 374 is outside of Victoria's bounds.
Starting check for geopoint # 375
	>Shape at data index 375 is outside of Victoria's bounds.
Starting check for geopoint # 376
	>Coordinates at data index 376 contains point
Starting check for geopoint # 377
	>Coordinates at data index 377 contains point
Starting check for geopoint # 378
	>Coordinates at data index 378 contains point
Starting check for geopoint # 379
	>Shape at data index 379 is outside of Victoria's bounds.
Starting check for geopoint # 380
Starting check for geopoint # 381
Starting check for geopoint # 382
Starting check for geopoint # 383
	>Shape

	>Coordinates at data index 570 contains point
Starting check for geopoint # 571
	>Shape at data index 571 is outside of Victoria's bounds.
Starting check for geopoint # 572
	>Coordinates at data index 572 contains point
Starting check for geopoint # 573
Starting check for geopoint # 574
Starting check for geopoint # 575
	>Coordinates at data index 575 contains point
Starting check for geopoint # 576
	>Shape at data index 576 is outside of Victoria's bounds.
Starting check for geopoint # 577
	>Coordinates at data index 577 contains point
Starting check for geopoint # 578
	>Coordinates at data index 578 contains point
Starting check for geopoint # 579
Starting check for geopoint # 580
Starting check for geopoint # 581
	>Coordinates at data index 581 contains point
Starting check for geopoint # 582
Starting check for geopoint # 583
Starting check for geopoint # 584
Starting check for geopoint # 585
	>Coordinates at data index 585 contains point
Starting check for geopoint # 586
Starting 

	>Coordinates at data index 782 contains point
Starting check for geopoint # 783
	>Coordinates at data index 783 contains point
Starting check for geopoint # 784
	>Shape at data index 784 is outside of Victoria's bounds.
Starting check for geopoint # 785
	>Coordinates at data index 785 contains point
Starting check for geopoint # 786
Starting check for geopoint # 787
	>Coordinates at data index 787 contains point
Starting check for geopoint # 788
Starting check for geopoint # 789
	>Coordinates at data index 789 contains point
Starting check for geopoint # 790
Starting check for geopoint # 791
Starting check for geopoint # 792
	>Coordinates at data index 792 contains point
Starting check for geopoint # 793
	>Shape at data index 793 is outside of Victoria's bounds.
Starting check for geopoint # 794
	>Shape at data index 794 is outside of Victoria's bounds.
Starting check for geopoint # 795
Starting check for geopoint # 796
Starting check for geopoint # 797
Starting check for geopoint # 7

...

	>Coordinates at data index 9194 contains point
Starting check for geopoint # 9195
	>Coordinates at data index 9195 contains point
Starting check for geopoint # 9196
	>Coordinates at data index 9196 contains point
Starting check for geopoint # 9197
Starting check for geopoint # 9198
	>Coordinates at data index 9198 contains point
Starting check for geopoint # 9199
	>Coordinates at data index 9199 contains point
Starting check for geopoint # 9200
	>Shape at data index 9200 is outside of Victoria's bounds.
Starting check for geopoint # 9201
Starting check for geopoint # 9202
	>Coordinates at data index 9202 contains point
Starting check for geopoint # 9203
	>Coordinates at data index 9203 contains point
Starting check for geopoint # 9204
	>Coordinates at data index 9204 contains point
Starting check for geopoint # 9205
Starting check for geopoint # 9206
	>Coordinates at data index 9206 contains point
Starting check for geopoint # 9207
	>Shape at data index 9207 is outside of Victoria's b

	>Coordinates at data index 9361 contains point
Starting check for geopoint # 9362
	>Coordinates at data index 9362 contains point
Starting check for geopoint # 9363
	>Shape at data index 9363 is outside of Victoria's bounds.
Starting check for geopoint # 9364
Starting check for geopoint # 9365
Starting check for geopoint # 9366
Starting check for geopoint # 9367
	>Coordinates at data index 9367 contains point
Starting check for geopoint # 9368
Starting check for geopoint # 9369
	>Coordinates at data index 9369 contains point
Starting check for geopoint # 9370
Starting check for geopoint # 9371
Starting check for geopoint # 9372
Starting check for geopoint # 9373
Starting check for geopoint # 9374
Starting check for geopoint # 9375
	>Shape at data index 9375 is outside of Victoria's bounds.
Starting check for geopoint # 9376
Starting check for geopoint # 9377
	>Shape at data index 9377 is outside of Victoria's bounds.
Starting check for geopoint # 9378
	>Coordinates at data index 9378 

	>Coordinates at data index 9571 contains point
Starting check for geopoint # 9572
	>Coordinates at data index 9572 contains point
Starting check for geopoint # 9573
	>Shape at data index 9573 is outside of Victoria's bounds.
Starting check for geopoint # 9574
	>Coordinates at data index 9574 contains point
Starting check for geopoint # 9575
Starting check for geopoint # 9576
Starting check for geopoint # 9577
	>Coordinates at data index 9577 contains point
Starting check for geopoint # 9578
Starting check for geopoint # 9579
	>Shape at data index 9579 is outside of Victoria's bounds.
Starting check for geopoint # 9580
Starting check for geopoint # 9581
	>Coordinates at data index 9581 contains point
Starting check for geopoint # 9582
Starting check for geopoint # 9583
Starting check for geopoint # 9584
Starting check for geopoint # 9585
	>Shape at data index 9585 is outside of Victoria's bounds.
Starting check for geopoint # 9586
Starting check for geopoint # 9587
Starting check for g

Starting check for geopoint # 9778
Starting check for geopoint # 9779
	>Coordinates at data index 9779 contains point
Starting check for geopoint # 9780
	>Shape at data index 9780 is outside of Victoria's bounds.
Starting check for geopoint # 9781
	>Coordinates at data index 9781 contains point
Starting check for geopoint # 9782
	>Coordinates at data index 9782 contains point
Starting check for geopoint # 9783
Starting check for geopoint # 9784
	>Coordinates at data index 9784 contains point
Starting check for geopoint # 9785
Starting check for geopoint # 9786
	>Shape at data index 9786 is outside of Victoria's bounds.
Starting check for geopoint # 9787
	>Shape at data index 9787 is outside of Victoria's bounds.
Starting check for geopoint # 9788
Starting check for geopoint # 9789
Starting check for geopoint # 9790
	>Coordinates at data index 9790 contains point
Starting check for geopoint # 9791
Starting check for geopoint # 9792
	>Shape at data index 9792 is outside of Victoria's bou

	>Coordinates at data index 9987 contains point
Starting check for geopoint # 9988
	>Shape at data index 9988 is outside of Victoria's bounds.
Starting check for geopoint # 9989
Starting check for geopoint # 9990
	>Coordinates at data index 9990 contains point
Starting check for geopoint # 9991
Starting check for geopoint # 9992
	>Shape at data index 9992 is outside of Victoria's bounds.
Starting check for geopoint # 9993
	>Shape at data index 9993 is outside of Victoria's bounds.
Starting check for geopoint # 9994
Starting check for geopoint # 9995
Starting check for geopoint # 9996
	>Shape at data index 9996 is outside of Victoria's bounds.
Starting check for geopoint # 9997
Starting check for geopoint # 9998
	>Coordinates at data index 9998 contains point
Starting check for geopoint # 9999

 POINTS FOUND: 3469
POINTS TOTAL: 10000
FINISHED IN: 0:00:20.574570


## Join coord data to results data using left join

In [ ]:
result_data = pandas.DataFrame(results)

In [ ]:
result_data.head(5)

In [ ]:
coord_data_index_list = list(coord_data.index)
coord_data_index_list

In [ ]:
coord_data = pandas.concat([coord_data, pandas.Series(coord_data_index_list).rename("row_index_new")], axis = 1)

In [ ]:
coord_data.head(10)

In [ ]:
final_data = pandas.merge(left = coord_data, 
                          right = result_data,
                          left_on = "row_index_new",
                          right_on = "data_row_index")

In [ ]:
final_data.head()

## Export coordinate data with index label

Data should be exported with data_index label to join to output files data_index column

In [ ]:
final_data.to_csv(export_path + "new_data.csv", index = False)